<a href="https://colab.research.google.com/github/cooper-hird/WAW-score-NCAAM/blob/main/All_Game_Scores_Page_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import math
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from datetime import timedelta
from zoneinfo import ZoneInfo



# Create a DatePicker with an initial date (e.g., today's date)
date_picker = widgets.DatePicker(
    description='Pick a Date',
    value= datetime.now(ZoneInfo("America/New_York")).date() - timedelta(days=1),  # Initial date
    disabled=False
)


class Game:
    def __init__(self, home_team, away_team, home_score, away_score, home_rank=400, away_rank=400, home_bigboard_score = 0, away_bigboard_score = 0, winner=None):
        self.home_team = home_team
        self.away_team = away_team
        self.home_score = home_score
        self.away_score = away_score
        self.home_rank = home_rank
        self.away_rank = away_rank
        self.home_bigboard_score = home_bigboard_score
        self.away_bigboard_score =  away_bigboard_score
        self.winner = winner
        self.upset_score = 0  # Placeholder for the calculated upset score
        self.composite_score = 0  # Placeholder for the calculated WAW score

    def calculate_composite_score(self, final_score_weight=0.75, ranking_score_weight=0.075, upset_score_weight=0.05, bigboard_score_weight = 0.125):
        final_score_factor = 1
        ranking_factor = 1
        upset_factor = 1
        bigboard_factor = 1

        # Closeness Score (difference in scores)
        closeness_score = max(0, 100 - (abs(self.home_score - self.away_score) * final_score_factor))

        # Rankings Score (average of home and away team ranks)
        rankings_score = max(0, min(100, 100 - (((self.home_rank + self.away_rank) / 2) * ranking_factor)))

        # Upset Score (if the lower-ranked team wins)
        upset_score = 0
        if self.winner == self.home_team:
            if self.home_rank > self.away_rank:  # Home team is ranked worse (higher number)
                upset_score = max(0, min(100, ((1/self.away_rank))* 100 * upset_factor))
        elif self.winner == self.away_team:
            if self.away_rank > self.home_rank:  # Away team is ranked worse (higher number)
                upset_score = max(0, min(100, (1/self.home_rank)* 100 * upset_factor))

        bigboard_score = ((self.home_bigboard_score + self.away_bigboard_score)/2) * bigboard_factor

        # Final composite score
        self.composite_score = round(
            (closeness_score * final_score_weight) +
            (rankings_score * ranking_score_weight) +
            (upset_score * upset_score_weight) +
            (bigboard_score * bigboard_score_weight), 1
        )

        return bigboard_score

class GameFetcher:
    def __init__(self, box_scores_url, rankings_url):
        self.box_scores_url = box_scores_url
        self.rankings_url = rankings_url

    def fetch_box_scores(self):
        response = requests.get(self.box_scores_url)
        games = []

        if 'text/html' in response.headers.get('content-type', ''):
            soup = BeautifulSoup(response.content, 'html.parser')
            game_pods = soup.find_all('div', class_='gamePod')

            for game in game_pods:
                teams = game.find_all('span', class_='gamePod-game-team-name')
                scores = game.find_all('span', class_='gamePod-game-team-score')
                ranks = game.find_all('span', class_='gamePod-game-team-rank')

                if len(teams) == 2 and len(scores) == 2:
                    home_team = teams[1].text.strip()
                    away_team = teams[0].text.strip()

                    home_rank = ranks[1].text.strip() or 'NR'
                    away_rank = ranks[0].text.strip() or 'NR'
                    try:
                      home_score = int(scores[1].text.strip())
                      away_score = int(scores[0].text.strip())
                    except ValueError:
                      continue
                    winner = home_team if home_score > away_score else (away_team if away_score > home_score else "Tie")

                    # Create a Game object and add it to the games list
                    game_record = Game(home_team, away_team, home_score, away_score, home_rank=home_rank, away_rank=away_rank, winner=winner)
                    games.append(game_record)
        else:
            print("Error: The response is not in HTML format or failed to fetch.")
        return games

    def fetch_rankings(self):
        response = requests.get(self.rankings_url)
        teams_rankings_dict = {}

        if 'text/html' in response.headers.get('content-type', ''):
            soup = BeautifulSoup(response.content, 'html.parser')
            rows = soup.find_all('div', class_='layout-content')
            td_data = []

            for row in rows:
                trs = row.find_all('tr')
                for tr in trs:
                    tds = tr.find_all('td')
                    for td in tds:
                        td_data.append(td.text.strip())

            grouped_data = [td_data[i:i+12] for i in range(0, len(td_data), 12)]
            for group in grouped_data:
                teams_rankings_dict[group[2]] = group[0]
        else:
            print("Error: The response is not in HTML format or failed to fetch.")
        return teams_rankings_dict

    def fetch_bigboard(self):
      response = requests.get('https://tankathon.com/big_board')

      if 'text/html' in response.headers.get('content-type', ''):
          soup = BeautifulSoup(response.content, 'html.parser')

          # Find all the pick numbers
          picks = soup.find_all('div', class_='mock-row-pick-number')

          # Find all the team names
          teams = soup.find_all('div', class_='mock-row-school-position')

          # Initialize lists to store picks and teams
          picks_array = []
          teams_array = []

          # Extract pick numbers
          for pick in picks:
              picks_array.append(int(pick.text.strip()))  # Convert to integers

          # Extract team names (split by ' | ' and take the second element)
          for team in teams:
              team_val = team.text.split(" | ")[1].strip()  # Strip any extra spaces or newlines
              teams_array.append(team_val)

          # Ensure picks and teams are correctly paired
          unique_entries = list(set(zip(picks_array, teams_array)))  # Remove duplicates
          unique_entries.sort(key=lambda x: x[0])  # Sort by pick number

          # Unpack into separate arrays
          picks_array, teams_array = zip(*unique_entries)

          # Determine the maximum pick value
          max_pick = max(picks_array)

          # Calculate raw pick scores
          raw_scores = {}
          for pick in sorted(picks_array, reverse=True):  # Start from the largest pick
              if pick == max_pick:
                  raw_scores[pick] = 1   # Largest pick gets a raw score of 1
              else:
                  raw_scores[pick] = raw_scores[pick + 1] * 1.1  # Exponential growth

          # Normalize scores to a 1-100 scale
          min_raw_score = min(raw_scores.values())
          max_raw_score = max(raw_scores.values())
          normalized_scores = {
              pick: 100 * (score - min_raw_score) / (max_raw_score - min_raw_score)
              for pick, score in raw_scores.items()
          }

          # Create a dictionary with team as the key and total score as the value
          team_scores = {}
          for pick, team in zip(picks_array, teams_array):
              team_scores[team] = team_scores.get(team, 0) + normalized_scores[pick]

          # Convert all scores to integers
          team_scores = {team: int(round(score)) for team, score in team_scores.items()}


      else:
          print("Error: The response is not in HTML format or failed to fetch.")
      return team_scores


# Main function to orchestrate the flow
def run_script(b):
    games_date = date_picker.value.strftime('%Y/%m/%d')
    box_scores_url = 'https://www.ncaa.com/scoreboard/basketball-men/d1/' + games_date + '/all-conf'
    rankings_url = 'https://www.ncaa.com/rankings/basketball-men/d1/ncaa-mens-basketball-net-rankings'
    bigboard_url = 'https://tankathon.com/big_board'

    # Instantiate the fetcher class
    game_fetcher = GameFetcher(box_scores_url, rankings_url)

    # Fetch box scores and rankings
    games = game_fetcher.fetch_box_scores()
    teams_rankings_dict = game_fetcher.fetch_rankings()
    bigboard = game_fetcher.fetch_bigboard()

    if not games:
        print("No games found.")
        return


    # Update the ranks of games using the fetched rankings
    for game in games:
        game.home_rank = int(teams_rankings_dict.get(game.home_team, 400))
        game.away_rank = int(teams_rankings_dict.get(game.away_team, 400))
        game.home_bigboard_score = int(bigboard.get(game.home_team,0))
        game.away_bigboard_score = int(bigboard.get(game.home_team,0))
        #Calculate the composite score for each game
        game.bigboard_score = ( game.calculate_composite_score() / 8) #Dividing by 10 because weight is .125

    # Sort the games by WAW score in descending order
    sorted_games = sorted(games, key=lambda g: g.composite_score, reverse=True)

    # Print the sorted games with their WAW scores
    for game in sorted_games:
        print(f"Home Team: {game.home_team} (Rank: {game.home_rank}) vs Away Team: {game.away_team} (Rank: {game.away_rank})")
        print(f"WAW Score: {game.composite_score}")
        print(f"Big Board Score: {game.bigboard_score}")
        #if(game.upset_score > 0):
       #   print(f"Upset Score: {game.upset_score}")
        print()


# Function to display the selected date
def on_date_change(change):
    print(f"Selected Date: {change.new}")

# Observe changes to the date picker and call the function
date_picker.observe(on_date_change, names='value')

# Create the button and link it to the run_script function
button = widgets.Button(description="Run Script")
button.on_click(run_script)

# Display the button in the notebook
display(date_picker)
display(button)